In [0]:
from google.colab import drive
import numpy as np
import pandas as pd
import gc
import os
import time
from datetime import datetime

In [2]:
try:
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
except:
    !pip install -U ckiptagger[tfgpu,gdown]
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [0]:
#set GPU accelerate
#Runtime > Change runtime type > Hardward accelerator > GPU

In [45]:
drive.mount('/content/drive/', force_remount=True)
exec_dt = datetime.strftime(datetime.now(), '%Y%m%d_%H%M%S')
path = '/content/drive/My Drive/side_project/20191109_ptt_analysis/'
neg_df = pd.read_table(path + 'ntusd-negative.txt', names=['word'])
pos_df = pd.read_table(path + 'ntusd-positive.txt', names=['word'])

Mounted at /content/drive/


In [5]:
#todo: change hatepolitics as example
for x in os.listdir(path):
    if 'pol_check' in x:
        print(x)
        pol_df = pd.read_csv(path + x)

Gossiping-37700-39294_1207_article_20191229_192300_pol_check_20191231_003120.csv


In [6]:
data_utils.download_data_gdown("./")

#CUDA_VISIBLE_DEVICES environment variable, e.g.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)

ws = WS("./data", disable_cuda=False)
#pos = POS("./data", disable_cuda=False)
#ner = NER("./data", disable_cuda=False)

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
1.88GB [00:14, 133MB/s] 


In [0]:
#ws(['我很不開心', '不快樂'])
#[['我', '很', '不', '開心'], ['不', '快樂']]

#todo: 
#1. check segement & non-segment percision by identified abnormal user
#2. strip counted string for calculating sentiment words

# get neo4j group

In [0]:
#currently community is in hatepolitics
gp_df = pd.read_csv(path + 'neo4j_trns_20191231_135302.csv')
gp_df.pagerank = gp_df.pagerank.astype(float)
gp_df['rnk'] = gp_df.groupby(['community'])['pagerank'].rank(method = 'dense', ascending=False)

# select part of DataFrame

In [0]:
pol_df['corpus'] = pol_df.article_title + ' ' + pol_df.content

In [0]:
#slc_df = art_df[art_df.author.isin(set(cand_usr))].copy()
start_idx = 1
end_idx = 20000
slc_id = set(gp_df[gp_df.rnk.between(start_idx, end_idx)].id)

cond_1 = pol_df.author.isin(slc_id)
cond_2 = pol_df.party != 'undef'

slc_df = pol_df[cond_1 & cond_2].copy()

In [16]:
slc_df.shape

(145, 18)

In [0]:
#selected 
#slc_idx = 200
#slc_df = pol_df[pol_df.party != 'undef'][:slc_idx]

In [18]:
#if memory insufficient, try to delete and garbage collect => run 10% data
print(time.ctime())
print(pol_df.shape)
cont_ws = ws(slc_df.corpus)
print(time.ctime())

Wed Jan  1 01:48:32 2020
(31206, 18)
Wed Jan  1 01:49:45 2020


In [19]:
sent_dict = { 
    'neg' : neg_df.word.to_list()
    , 'pos' : pos_df.word.to_list()
}

rs_lst = []

for idx, ws_item in enumerate(cont_ws):
    row = []
    row.append(ws_item)
    for sent in sent_dict:
        cnt = 0
        for word in sent_dict[sent]:
            cnt = cnt + ws_item.count(word)
        row.append(cnt)
    rs_lst.append(row)
    if idx % 500 == 1:
        print(idx)
        
sent_df = pd.DataFrame(rs_lst, columns=['ws_lst', 'neg', 'pos'])
sent_df['senti_sum'] = sent_df.pos - sent_df.neg

1


In [0]:
slc_df.reset_index(inplace=True, drop=True)

In [0]:
pol_sent_df = pd.concat([slc_df, sent_df], axis=1)

# personal political spectrum (this is step6)

In [0]:
pol_sent_df.loc[pol_sent_df.senti_sum > 0, 'senti_flag'] = 'pos' 
pol_sent_df.loc[pol_sent_df.senti_sum < 0, 'senti_flag'] = 'neg' 

In [0]:
pol_dict = {
 'deep_dpp_kw' : 'dpp',
 'light_dpp_kw' : 'dpp',
 'tsai_kw' : 'dpp',
 'han_kw' : 'kmt',
 'light_kmt_kw' : 'kmt',
 'deep_kmt_kw' : 'kmt',
 'third_party_kw' : '3rd',
 'unrelated_kw' : 'undef',
 'hk_protest' : 'event_hk',
 'china_spy' : 'event_chn'
 }

pol_sent_df['party_aggr'] = pol_sent_df.party.replace(pol_dict)

In [0]:
cond_kmt = pol_sent_df.party_aggr == 'kmt'
cond_dpp = pol_sent_df.party_aggr == 'dpp'
cond_3rd = pol_sent_df.party_aggr == '3rd'

cond_pos = pol_sent_df.senti_flag == 'pos'
cond_neg = pol_sent_df.senti_flag == 'neg'

pol_sent_df.loc[cond_kmt & cond_pos, 'party_aggr_sent'] = 'kmt'
pol_sent_df.loc[cond_dpp & cond_neg, 'party_aggr_sent'] = 'kmt'

pol_sent_df.loc[cond_dpp & cond_pos, 'party_aggr_sent'] = 'dpp'
pol_sent_df.loc[cond_kmt & cond_neg, 'party_aggr_sent'] = 'dpp'

pol_sent_df.loc[cond_3rd & cond_pos, 'party_aggr_sent'] = '3rd_pos'
pol_sent_df.loc[cond_3rd & cond_neg, 'party_aggr_sent'] = '3rd_neg'

In [0]:
pol_sent_aggr_df = pol_sent_df.groupby(['author', 'party_aggr_sent'], as_index=False)\
                   .agg({'article_id':'count'}).rename(columns={'article_id':'cnt'})

In [0]:
pol_sent_aggr_df['rnk'] = pol_sent_aggr_df.groupby(['author'])['cnt'].rank(method = 'dense', ascending=False)

In [0]:
cond_1 = pol_sent_aggr_df.rnk == 1
cond_2 = pol_sent_aggr_df.cnt != 1
pol_sent_aggr_df[cond_1 & cond_2].to_csv(path + 'pol_sent_aggr_{}.csv'.format(exec_dt))